In [86]:
import os
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta


def get_ts_daily_adjusted(instr):
    """Get data directly from Alpha Vantage"""
    r = requests.get('https://www.alphavantage.co/query', params={
        'function': 'TIME_SERIES_DAILY',  # Don't need their adjusted after all
        'symbol': instr,
        'apikey': os.environ['ALPHA_VANTAGE_API'],
        'outputsize': 'full',  # Alternative: compact
    })

    # Data comes with metadata that we don't need
    data = r.json()['Time Series (Daily)']

    df = pd.DataFrame.from_dict(data, orient='index')
    df = df.apply(pd.to_numeric)
    df.index = pd.to_datetime(df.index)
    
    # Because Alpha Vantage is offset incorrectly by 1, we need to fix it up
    df.index += timedelta(days=1)
    return df


def working_data(instr, target, lower, upper):
    """Creates an enriched dataset to work with"""
    df = get_ts_daily_adjusted(instr)
    try:
        target_date = datetime.strptime(target, '%d/%m/%Y')
    except ValueError as e:
        raise e

    # Actual dates we are interested in
    lower_date = target_date - timedelta(days=lower)
    upper_date = target_date + timedelta(days=upper)

    # Specs want us to call more than that
    lower_date_extreme = target_date - timedelta(days=(2 * lower + 1))
    upper_date_extreme = target_date + timedelta(days=(2 * upper))

    # Tighten to the range we want (and show non-trading days too)
    df = df.reindex(pd.date_range(lower_date_extreme, upper_date_extreme, freq='D')).fillna(method='ffill')
    #     df = df[(df.index >= lower_date_extreme) & (df.index <= upper_date_extreme)]

    # Tag with relative dates
    df = df.apply(tag_relative_date, axis=1, args=(target_date, lower_date, upper_date))

    #  Calculate the data we want
    df['Return'] = df['4. close'].diff()
    df['Return (%)'] = df['4. close'].pct_change()


    return df



def cm_return(df):
    i = 0
    cm = 0.0
    while i < len(df):
        val = df['Return'][i]
        # check to see if val is not nan
        if not np.isnan(val):
            percentage = df['Return (%)'][i]
            if percentage > 0:
                cm += val
            elif percentage < 0:
                cm -= val
        i += 1
    return cm


def av_return(df):
    return cm_return(df)/len(df)


def tag_relative_date(row, target, lower, upper):
    """Tags a row with it's relative distance from target date if we are interested in it"""
    row['Relative Date'] = (row.name - target).days if lower <= row.name <= upper else np.nan
    return row

def add_performance(df):
    df['CM Return'] = np.nan
    df['AV Return'] = np.nan

    for i in range(len(df)):
        if np.isnan(df.iloc[i]['Relative Date']):
            continue
        else:
            pass
    #         df['CM Return'].iloc[i] = df['Return'][i-lower:i+upper].sum()
            df['CM Return'].iloc[i] = df['4. close'].iloc[i+upper] - df['4. close'].iloc[i-lower]
            df['AV Return'].iloc[i] = df['CM Return'].iloc[i] / (lower + upper + 1)


In [81]:
instr = 'ABP.AX'
target = '10/12/2012'
lower = 3
upper = 5

In [15]:
instr = 'CBA.AX'
target = '09/01/2000'
lower = 5
upper = 5

In [87]:
df = working_data(instr, target, lower, upper)

In [88]:
df[['4. close', 'Relative Date', 'Return', 'Return (%)']]

,4. close,Relative Date,Return,Return (%)
2012-12-03,2.0420,NaN,NaN,NaN
2012-12-04,2.0122,NaN,-0.0298,-0.014594
2012-12-05,2.0122,NaN,0.0000,0.000000
2012-12-06,2.0420,NaN,0.0298,0.014810
2012-12-07,2.0420,-3.0,0.0000,0.000000
2012-12-08,2.0420,-2.0,0.0000,0.000000
2012-12-09,2.0420,-1.0,0.0000,0.000000
2012-12-10,2.0122,0.0,-0.0298,-0.014594
2012-12-11,2.0420,1.0,0.0298,0.014810
2012-12-12,2.0620,2.0,0.0200,0.009794


In [89]:
df['CM Return'] = np.nan
df['AV Return'] = np.nan

for i in range(len(df)):
    if np.isnan(df.iloc[i]['Relative Date']):
        continue
    else:
        pass
#         df['CM Return'].iloc[i] = df['Return'][i-lower:i+upper].sum()
        df['CM Return'].iloc[i] = df['4. close'].iloc[i+upper] - df['4. close'].iloc[i-lower]
        df['AV Return'].iloc[i] = df['CM Return'].iloc[i] / (lower + upper + 1)
df        

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Relative Date,Return,Return (%),CM Return,AV Return
2012-12-03,2.0420,2.0520,2.0321,2.0420,1.4754,489408.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
2012-12-04,2.0321,2.0371,2.0122,2.0122,1.4538,290924.0,0.0,1.0,NaN,-0.0298,-0.014594,NaN,NaN
2012-12-05,2.0122,2.0420,2.0122,2.0122,1.4538,381825.0,0.0,1.0,NaN,0.0000,0.000000,NaN,NaN
2012-12-06,2.0321,2.0420,2.0221,2.0420,1.4754,331801.0,0.0,1.0,NaN,0.0298,0.014810,NaN,NaN
2012-12-07,2.0420,2.0470,2.0221,2.0420,1.4754,311761.0,0.0,1.0,-3.0,0.0000,0.000000,0.0498,0.005533
2012-12-08,2.0420,2.0470,2.0221,2.0420,1.4754,311761.0,0.0,1.0,-2.0,0.0000,0.000000,0.0498,0.005533
2012-12-09,2.0420,2.0470,2.0221,2.0420,1.4754,311761.0,0.0,1.0,-1.0,0.0000,0.000000,0.0598,0.006644
2012-12-10,2.0420,2.0420,2.0122,2.0122,1.4538,263228.0,0.0,1.0,0.0,-0.0298,-0.014594,0.0598,0.006644
2012-12-11,2.0122,2.0420,2.0122,2.0420,1.4754,209039.0,0.0,1.0,1.0,0.0298,0.014810,0.0598,0.006644
2012-12-12,2.0620,2.0620,2.0271,2.0620,1.4898,584883.0,0.0,1.0,2.0,0.0200,0.009794,0.1096,0.012178


In [90]:
df[['4. close', 'Relative Date', 'Return', 'Return (%)', 'CM Return', 'AV Return']]

,4. close,Relative Date,Return,Return (%),CM Return,AV Return
2012-12-03,2.0420,NaN,NaN,NaN,NaN,NaN
2012-12-04,2.0122,NaN,-0.0298,-0.014594,NaN,NaN
2012-12-05,2.0122,NaN,0.0000,0.000000,NaN,NaN
2012-12-06,2.0420,NaN,0.0298,0.014810,NaN,NaN
2012-12-07,2.0420,-3.0,0.0000,0.000000,0.0498,0.005533
2012-12-08,2.0420,-2.0,0.0000,0.000000,0.0498,0.005533
2012-12-09,2.0420,-1.0,0.0000,0.000000,0.0598,0.006644
2012-12-10,2.0122,0.0,-0.0298,-0.014594,0.0598,0.006644
2012-12-11,2.0420,1.0,0.0298,0.014810,0.0598,0.006644
2012-12-12,2.0620,2.0,0.0200,0.009794,0.1096,0.012178


In [79]:
abp = get_ts_daily_adjusted(instr)

In [80]:
# abp[pd.to_datetime('2012-12-10')]
target_date = datetime.strptime(target, '%d/%m/%Y')
abp[abp.index == (target_date)]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
2012-12-10,2.042,2.042,2.0122,2.0122,1.4538,263228,0.0,1.0


In [68]:
r = requests.get('https://www.alphavantage.co/query', params={
        'function': 'TIME_SERIES_DAILY_ADJUSTED',  # Don't need their adjusted after all
        'symbol': instr,
        'apikey': os.environ['ALPHA_VANTAGE_API'],
        'outputsize': 'full',  # Alternative: compact
    })

# Data comes with metadata that we don't need
data = r.json()['Time Series (Daily)']


In [69]:
data['2012-12-10']

{'1. open': '2.0122',
 '2. high': '2.0420',
 '3. low': '2.0122',
 '4. close': '2.0420',
 '5. adjusted close': '1.4754',
 '6. volume': '209039',
 '7. dividend amount': '0.0000',
 '8. split coefficient': '1.0000'}

In [77]:
data['2012-12-09']

{'1. open': '2.0420',
 '2. high': '2.0420',
 '3. low': '2.0122',
 '4. close': '2.0122',
 '5. adjusted close': '1.4538',
 '6. volume': '263228',
 '7. dividend amount': '0.0000',
 '8. split coefficient': '1.0000'}

In [66]:
data

{'2018-03-16': {'1. open': '3.4700',
  '2. high': '3.5000',
  '3. low': '3.4600',
  '4. close': '3.4700',
  '5. volume': '1681462'},
 '2018-03-13': {'1. open': '3.4300',
  '2. high': '3.4600',
  '3. low': '3.4200',
  '4. close': '3.4400',
  '5. volume': '751543'},
 '2018-03-12': {'1. open': '3.4600',
  '2. high': '3.5000',
  '3. low': '3.4400',
  '4. close': '3.4600',
  '5. volume': '637981'},
 '2018-03-11': {'1. open': '3.4500',
  '2. high': '3.4800',
  '3. low': '3.4300',
  '4. close': '3.4600',
  '5. volume': '690239'},
 '2018-03-08': {'1. open': '3.4900',
  '2. high': '3.5000',
  '3. low': '3.4400',
  '4. close': '3.4800',
  '5. volume': '831492'},
 '2018-03-07': {'1. open': '3.4500',
  '2. high': '3.4900',
  '3. low': '3.4400',
  '4. close': '3.4500',
  '5. volume': '2770897'},
 '2018-03-06': {'1. open': '3.4400',
  '2. high': '3.4550',
  '3. low': '3.4100',
  '4. close': '3.4300',
  '5. volume': '420650'},
 '2018-03-05': {'1. open': '3.4500',
  '2. high': '3.4600',
  '3. low': '3